## Análise Das Filiais

In [17]:
import json
import os
from datetime import datetime
from IPython.display import clear_output

os.makedirs("dados", exist_ok=True)

path = "../bd/W_FILIAIS_202508281436.json"

with open(path, "r", encoding="utf-8") as f:
    dados = json.load(f)

if isinstance(dados, dict):
    # In case the JSON has a root key containing the list
    filiais = list(dados.values())[0]
else:
    filiais = dados

In [18]:
# Exibição
def mostrar_filial(filial):
    print(json.dumps(filial, indent=4, ensure_ascii=False))
    print("-" * 40)

def salvar_resultados(resultados, descricao):
    """Salva resultados em JSON na pasta 'dados' com nome automático."""
    if not resultados:
        print("Nenhum resultado para salvar.")
        return

    # gera nome de arquivo com base na descrição e timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_arquivo = f"{descricao}_{timestamp}.json"
    caminho = os.path.join("dados", nome_arquivo)

    with open(caminho, "w", encoding="utf-8") as f:
        json.dump(resultados, f, indent=4, ensure_ascii=False)

    print(f"\nResultados salvos em: {caminho}")

In [19]:
# Nova função: busca por estatísticas específicas
def buscar_por_estatisticas():
    print("\n=== PESQUISA POR ESTATÍSTICAS ESPECÍFICAS ===")
    
    # Captura todas as colunas disponíveis com base na primeira filial
    if not filiais:
        print("Nenhuma filial cadastrada.")
        return
    colunas_disponiveis = list(filiais[0].keys())
    print("Colunas disponíveis para filtro:")
    for c in colunas_disponiveis:
        print(f"- {c}")
    
    print("\nCondições disponíveis: =, >, <, >=, <=, contém")
    
    colunas = input("\nDigite os nomes das colunas a considerar (separadas por vírgula): ").split(",")
    colunas = [c.strip().upper() for c in colunas]

    condicoes = []
    for coluna in colunas:
        colunas_map = {c.upper(): c for c in colunas_disponiveis}
        if coluna not in [c.upper() for c in colunas_disponiveis]:
            print(f"Atenção: coluna '{coluna}' não existe. Ignorando.")
            continue

        coluna_real = colunas_map[coluna]

        operador = input(f"Digite a condição para a coluna '{coluna}' (=, >, <, >=, <=, contém): ")
        valor = input(f"Digite o valor de referência para '{coluna}': ")
        condicoes.append((coluna, operador, valor))

    resultados = []
    for filial in filiais:
        atende = True
        for coluna, operador, valor in condicoes:
            dado = filial.get(coluna)
            if dado is None:
                atende = False
                break

            # Comparações numéricas
            if operador in [">", "<", ">=", "<=", "="]:
                try:
                    dado_float = float(dado)
                    valor_float = float(valor)
                    if operador == ">" and not (dado_float > valor_float): atende = False
                    elif operador == "<" and not (dado_float < valor_float): atende = False
                    elif operador == ">=" and not (dado_float >= valor_float): atende = False
                    elif operador == "<=" and not (dado_float <= valor_float): atende = False
                    elif operador == "=" and not (dado_float == valor_float): atende = False
                except ValueError:
                    # Se não for número, compara como string
                    if operador == "=" and str(dado).upper() != valor.upper():
                        atende = False

            elif operador.lower() == "contém":
                if valor.upper() not in str(dado).upper():
                    atende = False

        if atende:
            resultados.append(filial)

    if resultados:
        print("\n--- Resultados encontrados ---")
        return resultados
    else:
        print("Nenhum resultado encontrado.")
        return


In [20]:
from IPython.display import clear_output

def menu():
    clear_output(wait=True)
    print("\n=== CONSULTA DE FILIAIS ===")
    print("1 - Listar todas as filiais")
    print("2 - Buscar por cidade")
    print("3 - Buscar por UF")
    print("4 - Buscar por código da filial")
    print("5 - Pesquisar por estatísticas específicas")
    print("0 - Sair")

    opcao = input("Escolha uma opção: ")
    return opcao

opcao = menu()

if opcao == "1":
    resultados = filiais
    # for filial in resultados:
    #     mostrar_filial(filial)
    salvar_resultados(resultados, "todas_filiais")

elif opcao == "2":
    cidade = input("Digite o nome da cidade: ").upper()
    resultados = [f for f in filiais if f.get("CIDADE", "").upper() == cidade]
    # for r in resultados:
    #     mostrar_filial(r)
    salvar_resultados(resultados, f"filiais_cidade_{cidade}")

elif opcao == "3":
    uf = input("Digite a UF: ").upper()
    resultados = [f for f in filiais if f.get("UF", "").upper() == uf]
    # for r in resultados:
    #     mostrar_filial(r)
    salvar_resultados(resultados, f"filiais_UF_{uf}")

elif opcao == "4":
    cod = input("Digite o código da filial: ")
    resultados = [f for f in filiais if f.get("COD_FILIAL") == cod]
    # for r in resultados:
    #     mostrar_filial(r)
    salvar_resultados(resultados, f"filial_codigo_{cod}")

elif opcao == "5":
    resultados = buscar_por_estatisticas()
    salvar_resultados(resultados, "pesquisa_estatisticas")

elif opcao == "0":
    print("Saindo...")
else:
    print("Opção inválida.")



=== CONSULTA DE FILIAIS ===
1 - Listar todas as filiais
2 - Buscar por cidade
3 - Buscar por UF
4 - Buscar por código da filial
5 - Pesquisar por estatísticas específicas
0 - Sair

=== PESQUISA POR ESTATÍSTICAS ESPECÍFICAS ===
Colunas disponíveis para filtro:
- FILIAL
- COD_FILIAL
- TIPO_FILIAL
- REGIAO
- REDE_LOJAS
- AREA_M2
- CEP
- ENDERECO
- BAIRRO
- CIDADE
- UF

Condições disponíveis: =, >, <, >=, <=, contém
Atenção: coluna '' não existe. Ignorando.

--- Resultados encontrados ---

Resultados salvos em: dados\pesquisa_estatisticas_20250904_181245.json
